In [57]:
import requests
from bs4 import BeautifulSoup
url ='https://flightaware.com/live/findflight?origin=KSEA&destination=KLAX'
response = requests.get(url)
page = response.text


In [58]:
response.status_code

200

In [ ]:
!pip install selenium
!pip install tqdm

In [59]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
from datetime import datetime 
import os
import dateutil.parser
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import random
chromedriver = "chromedriver.exe" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

#driver = webdriver.Chrome(chromedriver)

#driver.get(url)

In [60]:
options= Options()
#options.add_argument("--headless") # Runs Chrome in headless mode.
options.add_argument('--allow-no-sandbox-job') # Bypass OS security model
options.add_argument("--test-type")
options.add_argument('--disable-gpu')  # applicable to windows os only
options.add_argument('start-maximized') # 
options.add_argument('disable-infobars')
#options.add_argument("--disable-extensions")
options.add_argument("--disable-web-security")
options.add_argument("--allow-running-insecure-content")
options.add_argument("--safebrowsing-disable-extension-blacklist")
options.add_argument("--safebrowsing-disable-download-protection")
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_argument('--disable-bundled-ppapi-flash')
#options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36")

#PROXY = "185.21.156.3:41258" # IP:PORT or HOST:PORT
#options.add_argument('--proxy-server=%s' % PROXY)


#options.add_extension("C:/Users/Zaixing/Projects/Luther/Adblock-Plus_v3.1.crx")
options.add_extension("Adblock-Plus_v3.1.crx")

In [75]:
driver = webdriver.Chrome(chromedriver,chrome_options=options)

In [76]:
driver.get('https://www.whoishostingthis.com/tools/user-agent/')

In [77]:
driver.get('https://www.flightaware.com')

In [ ]:
#log in
driver.find_element_by_xpath('//*[@id="topWrapper"]/div[1]/nav/div/div[2]/a').click()
driver.find_element_by_xpath('//*[@id="popupLogin"]/div/div[2]/div/form/div[1]/input[1]').send_keys('johnlovesflying')
driver.find_element_by_xpath('//*[@id="popupLogin"]/div/div[2]/div/form/div[1]/input[2]').send_keys('2014852')
driver.find_element_by_xpath('//*[@id="popupLogin"]/div/div[2]/div/form/button').click()

In [ ]:
destination_list=['KLAX','KSFO','PANC','KDEN','KLAS','KPHX','KORD','KPDX','KDFW','KSJC']

In [ ]:
# define function to generate flightid
def get_flight(x):
    driver.get(x)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    len_flight=len(soup.find_all('table')[1].find_all('tr'))
    row_num=list(range(1,len_flight,2))
    la_flight=[]
    for i in row_num:
        fl=soup.find_all('table')[1].find_all('tr')[i].find_all('td')[1].text.strip()
        la_flight.append(fl)
    la_flight_uni = sorted(list(set(la_flight)))

    return la_flight_uni

In [ ]:
#get all unique flightid for destination airports above
city_url_list=[]
for i in destination_list:
    temp_list = get_flight("https://flightaware.com/live/findflight?origin=KSEA&destination="+i)
    city_url_list.append(temp_list)
    time.sleep(8)

In [ ]:
#combine all the flightid into one list
import itertools
flightid_list=list(itertools.chain.from_iterable(city_url_list))


In [ ]:
#pickle master flightid list
import pickle
with open('master_flightid_list.pickle', 'wb') as f:
    pickle.dump(flightid_list, f, pickle.HIGHEST_PROTOCOL)

### The following steps are to generate a list of all flights during the past three months

In [ ]:
driver.get('https://flightaware.com/live/flight/ABX1817/history/160')

In [ ]:
#generate a master list of all url for each flight each day
url_all_flight=[]
for i in flightid_list:
    driver.get('https://flightaware.com/live/flight/'+i+'/history/160')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table=soup.find(class_='prettyTable fullWidth tablesaw tablesaw-stack')
    rows=[row for row in table.find_all('a')]
    url_part=rows[0:len(rows):3]
    
    url_per_flight=[]
    for j in url_part:
        if 'Z/KSEA'in str(j):
            j= str(j).split('>')[0].replace("<a href=",'').replace('"','')
            url_per_flight.append('https://flightaware.com'+j)
    time.sleep(8)
    url_all_flight.append(url_per_flight)

In [ ]:
url_all_flight=list(itertools.chain.from_iterable(url_all_flight))

In [ ]:
# pickle master list url
with open('master_url_list.pickle', 'wb') as f:
    pickle.dump(url_all_flight, f, pickle.HIGHEST_PROTOCOL)

### The following steps are to scrape info from each web page

In [39]:
def find_class(s,c):
    try:
        return s.find_all(class_=c)[0].text.strip()
    except:
        return None

In [40]:

def get_info_page(url):

    #driver.get(url)

    #soup = BeautifulSoup(driver.page_source, 'html.parser')
    soup = BeautifulSoup(url, 'html.parser')
    data={}
    #find airline
    ##airline=soup.find('title').text.split()[0]
    airline=soup.find(class_="flightPageAirlineCallsign").text.strip()
    
    #find flightid
    ##flightid=soup.find('title').text.split()[2].replace('(','').replace(')','')+soup.find('title').text.split()[3].replace('#','')
    flightid=soup.find(class_="flightPageIdent").text.strip().split('/')[0]
    
    #find flight date
    ##date=dateutil.parser.parse(soup.find('title').text.split()[5])
    date=dateutil.parser.parse(soup.find(class_='flightPageSummaryDepartureDay').text.strip().split(' ')[1])
    
    #find destination airport
    ##destination=soup.find('title').text.split()[9]
    destination=soup.find(class_="flightPageSummaryDestination").findChildren()[0].text.strip()
    
    #find departure gate
    ##gate=soup.find_all('strong')[3].text.split()[1]
    gate=find_class(soup,"flightPageAirportGate")
    
    # identify flight time data structure
    time_type = [i for i in soup.find_all(class_="flightPageDataTableHeading") if ('Departure Times' in i.text or 'Flight Times' in i.text)][0].text.strip()
    
    #find departure times based on data structure
    if time_type=='Departure Times':
        #find scheduled  Departure time
        scheduled_departure=soup.find_all(text="Scheduled ")[0].findNextSibling().text.strip()
    
        #find actual departure time
        actual_departure=soup.find_all(class_='flightPageDataActualTimeText')[0].text.strip()

        #find scheduled  takeoff time
        scheduled_takeoff=soup.find_all(text="Scheduled ")[0].findNextSibling().text.strip()
        
        #find actual takeoff time
        actual_takeoff=soup.find_all(class_='flightPageDataActualTimeText')[1].text.strip()

        
    if time_type=='Flight Times':
        #find scheduled  Departure time
        scheduled_departure=None
    
        #find actual departure time
        actual_departure=None

        #find scheduled  Departure time
        scheduled_takeoff=soup.find_all(class_='flightPageDataAncillaryText')[0].text.strip().split('\n')[1]
        
        #find actual takeoff time
        actual_takeoff=soup.find_all(class_='flightPageDataActualTimeText')[0].text.strip()

    
    #find aircraft type
    aircraft=soup.find(class_='flightPageData').text.strip().split('\xa0')[0]
    
    #find taxi time
    #taxi_time=soup.find(text="Taxi Time: ").findNextSibling().text.strip()
    
    #find delay status
    delay=soup.find(class_='flightPageOriginDelayStatus').text.strip().replace('(','').replace(')','')
    
    #find distance
    distance =''.join([i.text.strip() for i in soup.find_all(class_='flightPageData') if ('Actual:' in i.text or 'Planned:' in i.text or'Direct:' in i.text)])
    
    data[date]=[date,airline,aircraft,flightid,destination,gate,distance,
                scheduled_departure,actual_departure,scheduled_takeoff,actual_takeoff,delay]
    df=pd.DataFrame(data).T
    return df

In [74]:
import pickle

with open('master_url_list.pickle', 'rb') as f:
    url_all_flight=pickle.load(f)
len(url_all_flight)

with open('soup_bowl.pickle', 'rb') as f:
    soup_bowl=pickle.load(f)
len(soup_bowl)


103

In [ ]:
# select URLS not have been scraped
from random import shuffle

with open('url_toscrape.pickle', 'rb') as f:
    url_toscrape=pickle.load(f)
url_toscrape_list = list(url_toscrape.sample(frac=1)['url'])
len(url_toscrape_list)



In [ ]:
# start scraping!

#all_flight=[]
#soup_bowl = []
success_url = []
fail_url = []
n=1

for i in tqdm(url_toscrape_list[104:]):
    user_agent=random.choice(user_agent_list) # pick a user agent
    #PROXY = random.choice(useable_proxies[0:10]) # pick a proxy
    options.add_argument("user-agent="+user_agent)
    #options.add_argument('--proxy-server=%s' % PROXY)
    
    try:
        driver.get(i)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        soup_bowl.append(str(soup))
        #all_flight.append(get_info_page(i))
        success_url.append(i)
    except:
        fail_url.append(i)
    
    n+=1
    if n==50:
        with open('soup_bowl.pickle', 'wb') as fp:
            pickle.dump(soup_bowl, fp)
        n=1
 
    #result = pd.concat(all_flight)

        
    time.sleep(random.sample(range(9),1)[0])
    
    


In [79]:
len(soup_bowl)


352

In [80]:
len(success_url)

249

In [81]:
len(fail_url)

25

In [16]:
remaining_url = [i for i in url_toscrape['url'] if i not in success_url]
len(remaining_url)

5980

In [82]:
result = pd.concat([get_info_page(i) for i in soup_bowl])

AttributeError: 'NoneType' object has no attribute 'text'

In [18]:
driver.get('https://flightaware.com/live/flight/ASA348/history/20180625/0353Z/KSEA/KSJC')
soup = BeautifulSoup(driver.page_source, 'html.parser')
# scheduled_departure=soup.find_all(text="Scheduled ")[1].findNextSibling().text.strip()
# actual_departure=soup.find_all(class_='flightPageDataActualTimeText')[1].text.strip()
#print(soup.find('div', {'class': 'flightPageDataActualTimeText'}).findChildren())
soup=BeautifulSoup(str(soup), 'html.parser')


In [45]:
dateutil.parser.parse(soup.find(class_='flightPageSummaryDepartureDay').text.strip().split(' ')[1])

datetime.datetime(2018, 6, 24, 0, 0)

In [46]:
soup.find_all(class_='flightPageAirportGates')[0].text.strip()

"left Gate N9\n\n\nSeattle-Tacoma Intl - \nSEA\n\n\n\n\n\n\narrived at \xa0Gate 28\n\n\n\nSan Jose Int'l - \nSJC"

## Use proxy in requests

In [ ]:
import requests
url = 'https://httpbin.org/ip'
proxies = {
    "http": 'http://5.101.203.50:8080', 
    "https": 'http://5.101.203.50:8080'
}
response = requests.get(url,proxies=proxies)
print(response.json())

In [ ]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:20]:
        if i.xpath('.//td[3][contains(text(),"US")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [ ]:
proxies = get_proxies()
print(proxies)

In [ ]:
import requests
from itertools import cycle
import traceback
#If you are copy pasting proxy ips, put in the list below
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
proxies = get_proxies()
proxy_pool = cycle(proxies)
useable_proxies = []

url = 'https://httpbin.org/ip'
for i in range(1,31):
    #Get a proxy from the pool
    proxy = next(proxy_pool)
    print("Request #%d"%i)
    try:
        response.time = requests.get(url,proxies={"http": proxy, "https": proxy}).elapsed.total_seconds()
        if response.time<=4:
            useable_proxies.append(proxy)
        #print(response.time)
        #useable_proxies = useable_proxies.append(proxy)
    except:
        #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
        #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
        print("Skipping. Connnection error")
    

In [ ]:
random.choice(list(proxies))

In [ ]:
useable_proxies

In [ ]:
useable_proxies = []
useable_proxies.append(next(proxy_pool))
print(useable_proxies)

In [ ]:
useable_proxies

## Hide user agent

In [ ]:
import urllib.request
url = 'https://httpbin.org/user-agent'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
request = urllib.request.Request(url,headers={'User-Agent': user_agent})
response = urllib.request.urlopen(request)
html = response.read()
print(html)

In [12]:
import urllib.request
import random
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]
url = 'https://httpbin.org/user-agent'
#Lets make 5 requests and see what user agents are used 
for i in range(1,6):
    #Pick a random user agent
    user_agent = random.choice(user_agent_list)
    #Set the headers 
    headers = {'User-Agent': user_agent}
    #Make the request
    request = urllib.request.Request(url,headers={'User-Agent': user_agent})
    response = urllib.request.urlopen(request)
    html = response.read()
    
    print("Request #%d\nUser-Agent Sent:%s\nUser Agent Recevied by HTTPBin:"%(i,user_agent))
    print(html)
    print("-------------------\n\n")

Request #1
User-Agent Sent:Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko
User Agent Recevied by HTTPBin:
b'{"user-agent":"Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko"}\n'
-------------------


Request #2
User-Agent Sent:Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36
User Agent Recevied by HTTPBin:
b'{"user-agent":"Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36"}\n'
-------------------


Request #3
User-Agent Sent:Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36
User Agent Recevied by HTTPBin:
b'{"user-agent":"Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36"}\n'
-------------------


Request #4
User-Agent Sent:Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) C

In [ ]:
random.choice(user_agent_list)

In [ ]:
random.choice(useable_proxies)